In [1]:

%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'/app')
sys.path.insert(0,'..')

In [2]:
import nest_asyncio
nest_asyncio.apply()

In [129]:

from audio.redis import *
from audio.audio import *
#from audio.process import *

from pathlib import Path
import json

import uuid

import pandas as pd

import asyncio
import redis.asyncio as aioredis
from dataclasses import dataclass
import numpy as np

import json
import time

In [130]:
async def get_next_chunk_start(redis_inner_client, audio_name, length,shift):
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    diarization_result = diarization.data

    if len(diarization_result)>0:
        last_speech = diarization_result[-1]

        ended_silence = length - last_speech['end']
        print(ended_silence)
        if ended_silence<1:
            print('interrupted')
            return last_speech['start']+shift
        

        else:
            print('non-interrupted') 
            return last_speech['end']+shift



    else: return None

In [4]:
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'
import subprocess
from pydub import AudioSegment
import io
from time import sleep

from audio.redis import Transcript

In [5]:
redis_inner_client = await get_inner_redis()
redis_stream_client = await get_stream_redis()
client_id = '851f343e-4954-4f0a-8835-9664cc91c181'

In [7]:
async def diarize(client_id, audio_name, shift):
    await redis_inner_client.lpush('Audio2DiarizeQueue', f'{audio_name}:{client_id}')
    done = await redis_inner_client.brpop(f'DiarizeReady:{audio_name}')
    diarization = Diarisation(audio_name, redis_inner_client)
    await diarization.get()
    df = pd.DataFrame(diarization.data)
    df['len'] = df['end'] - df['start']

    if len(df)>0:
        expanded_df = pd.DataFrame(columns=['speaker', 'time'])
        for index, row in df.iterrows():
            time_range = np.arange(row['start'] * 100, row['end'] * 100, 1).astype(int)

            temp_df = pd.DataFrame({
                'speaker': row['speaker'],
                'time': time_range
            })
            expanded_df = pd.concat([expanded_df, temp_df], ignore_index=True)
        expanded_df['time'] = (expanded_df['time'] / 100.0).astype('float')
        expanded_df['conv_time'] = (expanded_df['time'] + shift).astype('float')

        return expanded_df.sort_values("time")

In [8]:

async def transcribe(audio_name):
    await redis_inner_client.lpush('Audio2TranscribeQueue', f'{audio_name}:{client_id}')
    _,done = await redis_inner_client.brpop(f'TranscribeReady:{audio_name}')
    transcription =  Transcript(audio_name,redis_inner_client)
    await transcription.get()
    df =  pd.concat([pd.DataFrame(t) for t in transcription.data])


    df['start'] = df['start'].astype('float')
    return df.sort_values("start")

In [9]:
await get_connections_df('initialFeed_audio',redis_stream_client)

,queue,length
0,initialFeed_audio:28431457-da74-4e02-9dac-ccbf...,3
1,initialFeed_audio:cfee46c2-db72-472a-bdd1-1de4...,3
2,initialFeed_audio:f1cd898e-1b08-4248-a9fd-2b12...,2
3,initialFeed_audio:6ce3d44e-4eda-4e5c-9a8b-6ea4...,16
4,initialFeed_audio:91c1695d-e594-4612-aca5-a276...,5
5,initialFeed_audio:91c1695d-e594-4612-aca5-a276...,6
6,initialFeed_audio:8acec099-2fe3-4fb1-a9a3-46f3...,2
7,initialFeed_audio:undefined,10
8,initialFeed_audio:c243960c-3e6f-4be2-abcd-c971...,2
9,initialFeed_audio:303509db-9efc-4b1f-a5ec-82fd...,2


In [10]:
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]


connections =  await get_connections('initialFeed_audio',redis_stream_client)


In [11]:
#[await redis_stream_client.delete(c) for c in connections]

In [12]:
connection_id = 'david_audio'

In [13]:

path = f'/app/testdata/{connection_id}.webm'

In [14]:
on_start = False

In [15]:

#on_start = True

In [155]:
start = 0
step = 30
max_length=600

In [156]:
redis_stream_client = await get_stream_redis()
redis_inner_client = await get_inner_redis()
connections =  await get_connections('initialFeed_audio',redis_stream_client)
connection_ids = [c.replace('initialFeed_audio:','') for c in connections]
start = await redis_inner_client.rpop(f'Start:{connection_id}')
start = float(start) if start else 0

#connection_id??
await writestream2file(connection_id,redis_stream_client)
print(start)
if on_start:  audio_slicer = await AudioSlicer.from_file(path,format = 'webm')
else       :  audio_slicer = await AudioSlicer.from_ffmpeg_slice(path,start,max_length)

slice_duration = audio_slicer.audio.duration_seconds
print(slice_duration)

if slice_duration > step:

    audio_data = await audio_slicer.export_data()
    audio_name = str(uuid.uuid4())
    audio = Audio(chunk_name=audio_name, redis_client=redis_inner_client, data=audio_data)
    await audio.save()

diarization_result, transcription_result = await asyncio.gather(
        diarize(client_id, audio_name, start),
        transcribe(audio_name)
    )

df = pd.merge_asof(transcription_result,diarization_result,left_on = 'start',right_on='time',direction='nearest')
df['speaker_change'] = df['speaker'] != df['speaker'].shift()
df['silence'] = df['start']-df['end'].shift()
df['speaker_change'] = np.where(df['silence']>2,True,df['speaker_change'])
df['speaker_change'] = df['speaker_change'].cumsum()
df = df.groupby('speaker_change').agg({'speaker': 'first', 'start': 'first', 'end': 'last'}).join(df.groupby('speaker_change').apply(lambda x:''.join(x['word'])).to_frame('text'))
df['len'] = df['end'] - df['start']
df['speaker'] = np.where(df['len']>0.5,df['speaker'],np.nan)
df['speaker'] = df['speaker'].fillna(method='ffill').fillna(method='bfill')
df['conv_start'] = df['start']+start
df['conv_end'] = df['end']+start

start_ = await get_next_chunk_start(redis_inner_client, audio_name, slice_duration,start)
start = start_ if start_ else start+slice_duration
await redis_inner_client.lpush(f'Start:{connection_id}', start)


await redis_inner_client.lpush(f'Segment:{connection_id}', json.dumps(df.to_dict(orient='records')))


219.38879456706283
600.048
None


In [154]:
df

,speaker,start,end,text,len,conv_start,conv_end
speaker_change,,,,,,,
1,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,18.29,22.79,Sorry about that. Agenda in-house. We'll get ...,4.50,178.756893,183.256893
2,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,25.01,28.13,"Thanks, James. We'll get decisions.",3.12,185.476893,188.596893
3,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,31.43,38.15,"We'll get actions. Now, this stuff I'm guessi...",6.72,191.896893,198.616893
4,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,38.55,38.91,"Now,",0.36,199.016893,199.376893
5,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,41.58,50.34,"when I look at the minutes, we can probably a...",8.76,202.046893,210.806893
6,dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,53.83,59.89,There may be other things that get captured. ...,6.06,214.296893,220.356893


In [150]:
diarization_result['speaker'].unique()

array(['dcde8443-9ac1-4a61-a1b6-7cd22dc9985b',
       '577fe873-b083-49ab-bde6-581670a11dde'], dtype=object)

In [153]:
diarization_result.groupby('speaker').count()

,time,conv_time
speaker,,
577fe873-b083-49ab-bde6-581670a11dde,69,69
dcde8443-9ac1-4a61-a1b6-7cd22dc9985b,2481,2481
